In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from mydatools.plot import plot_multiclass_feature_dist
from mydatools.features_analyze import get_top_k_corr

% matplotlib inline

## Config

In [2]:
trn_path = './data/input/application_train.csv'
tst_path = './data/input/application_test.csv'
id_col = 'SK_ID_CURR'
label_col = 'TARGET'

submission_path = './data/output/submission/submission.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
trn_df = pd.read_csv(trn_path)
trn_df['ds_type'] = 'trn'
trn_df[label_col] = trn_df[label_col].astype('int')

tst_df = pd.read_csv(tst_path)
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])

del(trn_df)
del(tst_df)

full_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,ds_type
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,trn
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,trn
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn


In [4]:
# dataset type
is_trn = full_df['ds_type'] == 'trn'
is_tst = full_df['ds_type'] == 'tst'

## Features

In [5]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])

**numerical_features**

In [6]:
numerical_features = full_df.dtypes[full_df.dtypes != 'object'].index.tolist()
numerical_features = [c for c in numerical_features if c not in [id_col, label_col, 'ds_type']]
add_features(numerical_features)

**categorical_features**

In [7]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
full_df[categorical_features].nunique()

CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
ds_type                        2
dtype: int64

In [8]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
categorical_features = categorical_features[:-1] # 为了去掉ds_type
# 去掉分类数太多
# categorical_features = full_df[categorical_features].columns[full_df[categorical_features].nunique() <= 10]

# get dummies
full_df = pd.get_dummies(full_df, columns=categorical_features, prefix_sep=':')

# add features
new_features = full_df.columns.tolist()[full_df.columns.tolist().index('ds_type') + 1:]
add_features(new_features)

### add extra data

In [9]:
def summary_extra_data(df, suffix):
    # 分类特征get_dummies
    df = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object'].tolist())
    df = df.fillna(0)

    # 求平均
    df = pd.concat([
        df.groupby('SK_ID_CURR').mean(),
        df.groupby('SK_ID_CURR').size().rename('cnt'),
    ], axis=1)

    # 给特征加上前缀
    df.columns = [suffix+f for f in df.columns.tolist()]

    return df

In [10]:
# bureau = pd.read_csv('./data/input/bureau.csv')
# test_df = bureau.iloc[:100, :5]
# test_df.head()

# summary_extra_data(test_df, 'test')

**bureau.csv**

In [11]:
bureau = pd.read_csv('./data/input/bureau.csv')
bureau = bureau.drop('SK_ID_BUREAU', axis=1)

bureau = summary_extra_data(bureau, 'bureau_')

full_df = full_df.join(bureau, on='SK_ID_CURR')
add_features(bureau.columns.tolist())

**previous_application.csv**

In [12]:
prevapp = pd.read_csv('./data/input/previous_application.csv')
prevapp = prevapp.drop('SK_ID_PREV', axis=1)

prevapp = summary_extra_data(prevapp, 'prevapp_')

full_df = full_df.join(prevapp, on='SK_ID_CURR')
add_features(prevapp.columns.tolist())

**POS_CASH_balance.csv**

In [13]:
pcblc = pd.read_csv('./data/input/POS_CASH_balance.csv')
pcblc = pcblc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1)

pcblc = summary_extra_data(pcblc, 'pcblc_')

full_df = full_df.join(pcblc, on='SK_ID_CURR')
add_features(pcblc.columns.tolist())

**credit_card_balance.csv**

In [14]:
ccblc = pd.read_csv('./data/input/credit_card_balance.csv')
ccblc = ccblc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1)

ccblc = summary_extra_data(ccblc, 'ccblc_')

full_df = full_df.join(ccblc, on='SK_ID_CURR')
add_features(ccblc.columns.tolist())

**installments_payments.csv**

In [15]:
installpay = pd.read_csv('./data/input/installments_payments.csv')
installpay = installpay.drop(['SK_ID_PREV'], axis=1)

installpay = summary_extra_data(installpay, 'installpay_')

full_df = full_df.join(installpay, on='SK_ID_CURR')
add_features(installpay.columns.tolist())

## LightGBM

In [16]:
trn_df = full_df[is_trn]
tst_df = full_df[is_tst]

X = trn_df[feature_columns]
y = trn_df[label_col]
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.20, random_state=20180521)

X_tst = tst_df[feature_columns]

d_trn = lgb.Dataset(X_trn.values, y_trn)
d_val = lgb.Dataset(X_val.values, y_val)

In [17]:
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 12,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
bst = lgb.train(params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=20)

[1]	training's auc: 0.699566	valid_1's auc: 0.702609
Training until validation scores don't improve for 20 rounds.
[2]	training's auc: 0.712178	valid_1's auc: 0.716399
[3]	training's auc: 0.716643	valid_1's auc: 0.721388
[4]	training's auc: 0.717158	valid_1's auc: 0.721833
[5]	training's auc: 0.721429	valid_1's auc: 0.725602
[6]	training's auc: 0.724169	valid_1's auc: 0.729552
[7]	training's auc: 0.725207	valid_1's auc: 0.730787
[8]	training's auc: 0.72896	valid_1's auc: 0.733464
[9]	training's auc: 0.731298	valid_1's auc: 0.735422
[10]	training's auc: 0.731585	valid_1's auc: 0.735659
[11]	training's auc: 0.731163	valid_1's auc: 0.735271
[12]	training's auc: 0.730942	valid_1's auc: 0.734951
[13]	training's auc: 0.731373	valid_1's auc: 0.73534
[14]	training's auc: 0.732139	valid_1's auc: 0.73663
[15]	training's auc: 0.733649	valid_1's auc: 0.738084
[16]	training's auc: 0.733778	valid_1's auc: 0.738188
[17]	training's auc: 0.733718	valid_1's auc: 0.738225
[18]	training's auc: 0.73446	val

[152]	training's auc: 0.781631	valid_1's auc: 0.775353
[153]	training's auc: 0.781739	valid_1's auc: 0.775454
[154]	training's auc: 0.781935	valid_1's auc: 0.775545
[155]	training's auc: 0.782099	valid_1's auc: 0.77566
[156]	training's auc: 0.78224	valid_1's auc: 0.775687
[157]	training's auc: 0.782371	valid_1's auc: 0.775667
[158]	training's auc: 0.782521	valid_1's auc: 0.775786
[159]	training's auc: 0.782646	valid_1's auc: 0.775882
[160]	training's auc: 0.78282	valid_1's auc: 0.77589
[161]	training's auc: 0.782958	valid_1's auc: 0.775917
[162]	training's auc: 0.783099	valid_1's auc: 0.775983
[163]	training's auc: 0.783281	valid_1's auc: 0.776085
[164]	training's auc: 0.783412	valid_1's auc: 0.776122
[165]	training's auc: 0.783509	valid_1's auc: 0.776193
[166]	training's auc: 0.783646	valid_1's auc: 0.776309
[167]	training's auc: 0.78376	valid_1's auc: 0.776324
[168]	training's auc: 0.783903	valid_1's auc: 0.776284
[169]	training's auc: 0.784035	valid_1's auc: 0.7763
[170]	training's 

[302]	training's auc: 0.798457	valid_1's auc: 0.781247
[303]	training's auc: 0.798523	valid_1's auc: 0.781226
[304]	training's auc: 0.798596	valid_1's auc: 0.78124
[305]	training's auc: 0.798696	valid_1's auc: 0.781224
[306]	training's auc: 0.798768	valid_1's auc: 0.78123
[307]	training's auc: 0.79884	valid_1's auc: 0.781237
[308]	training's auc: 0.798903	valid_1's auc: 0.781245
[309]	training's auc: 0.799	valid_1's auc: 0.781252
[310]	training's auc: 0.799072	valid_1's auc: 0.781255
[311]	training's auc: 0.799134	valid_1's auc: 0.781269
[312]	training's auc: 0.799268	valid_1's auc: 0.781292
[313]	training's auc: 0.799388	valid_1's auc: 0.781271
[314]	training's auc: 0.799466	valid_1's auc: 0.781243
[315]	training's auc: 0.799544	valid_1's auc: 0.781268
[316]	training's auc: 0.79965	valid_1's auc: 0.78131
[317]	training's auc: 0.79972	valid_1's auc: 0.781336
[318]	training's auc: 0.799777	valid_1's auc: 0.781331
[319]	training's auc: 0.79981	valid_1's auc: 0.781313
[320]	training's auc

[453]	training's auc: 0.80998	valid_1's auc: 0.783371
[454]	training's auc: 0.810052	valid_1's auc: 0.783345
[455]	training's auc: 0.810141	valid_1's auc: 0.783411
[456]	training's auc: 0.810202	valid_1's auc: 0.783437
[457]	training's auc: 0.81026	valid_1's auc: 0.783449
[458]	training's auc: 0.8103	valid_1's auc: 0.783441
[459]	training's auc: 0.810364	valid_1's auc: 0.783473
[460]	training's auc: 0.810411	valid_1's auc: 0.783431
[461]	training's auc: 0.810459	valid_1's auc: 0.78344
[462]	training's auc: 0.810504	valid_1's auc: 0.783453
[463]	training's auc: 0.810625	valid_1's auc: 0.783512
[464]	training's auc: 0.810709	valid_1's auc: 0.783503
[465]	training's auc: 0.810747	valid_1's auc: 0.783516
[466]	training's auc: 0.810817	valid_1's auc: 0.783498
[467]	training's auc: 0.810906	valid_1's auc: 0.783543
[468]	training's auc: 0.810939	valid_1's auc: 0.783541
[469]	training's auc: 0.811003	valid_1's auc: 0.783524
[470]	training's auc: 0.811075	valid_1's auc: 0.783561
[471]	training'

**tuning log**

* original

```
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 5,
    'num_leaves': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
```

* max_depth, num_leaves

```
4, 12: 0.812, 0.784 *
5, 20: 0.817, 0.783
6, 30: 0.850, 0.783
7, 42: 0.847, 0.783
```

In [18]:
# param tuning
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 12,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
tuning_param_dict = {
#     'max_depth': [3,4,5],
#     'num_leaves': [10,20,30],
#     'bagging_freq': [1,3,5,10],
    'reg_alpha': [0.1, 1, 10],
}

for p in ParameterGrid(tuning_param_dict):
    cur_params = params.copy()
    cur_params.update(p)
    print(p)
    bst = lgb.train(cur_params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=20, verbose_eval=0)
    print('train:', bst.best_score['training']['auc'], ', val:', bst.best_score['valid_1']['auc'])

{'reg_alpha': 0.1}
train: 0.8131078405524543 , val: 0.7836738891031625
{'reg_alpha': 1}
train: 0.8038277568094492 , val: 0.7828195916340148
{'reg_alpha': 10}
train: 0.8114419097802309 , val: 0.7842159411937176


In [19]:
# bst = lgb.train(params, trn_lgb, 1000, valid_sets=[trn_lgb, val_lgb], early_stopping_rounds=20,
#                 init_model=bst, 
#                 learning_rates=lambda iter: 0.1 * (0.99 ** iter))

In [20]:
imp_df = pd.DataFrame([bst.feature_importance()], columns=feature_columns, index=['importance']).T.sort_values(by='importance', ascending=False)
imp_df

,importance
EXT_SOURCE_3,215
EXT_SOURCE_2,196
EXT_SOURCE_1,177
DAYS_BIRTH,132
pcblc_CNT_INSTALMENT_FUTURE,127
AMT_ANNUITY,121
AMT_CREDIT,116
AMT_GOODS_PRICE,95
installpay_AMT_PAYMENT,89
DAYS_EMPLOYED,83


## predict

In [21]:
res_df = pd.DataFrame(bst.predict(X_tst), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)